In [0]:
%pip install 'splink[spark]'

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/317.3 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/317.3 MB 3.0 MB/s eta 0:01:47
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/317.3 MB 46.1 MB/s eta 0:00:07
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/317.3 MB 65.5 MB/s eta 0:00:05
     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/317.3 MB 109.1 MB/s eta 0:00:03
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/317.3 MB 111.4 MB/s eta 0:00:03
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/317.3 MB 111.1 MB/s eta 0:00:03
     ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/317.3 MB 103.3 MB/s eta 0:00:03
     ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/317.3 MB 102.4 MB/s eta 0:00:03
     ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/317.3 MB 107.5 MB/s eta 0:00:03
     ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/317.3 MB 112.4 MB/s eta 0:0

In [0]:
# 👾 IMPORTS 👾
from splink.spark.jar_location import similarity_jar_location
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, types
import pandas as pd
import string
from splink.spark.linker import SparkLinker
import splink.spark.comparison_library as cl
import splink.spark.comparison_template_library as ctl
from splink.spark.blocking_rule_library import block_on
import splink.comparison_helpers as ch
import splink.duckdb.comparison_template_library as clt
conf = SparkConf()

path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("/tmp_checkpoints/r1")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
%sql
USE CATALOG development;
USE SCHEMA splink_training_data;

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

###### Intent: 
We are training a `probabilistic record linkage` model to identify records that refer to the same person across different datasets or databases. 

This model will be used to reduce duplicates and consolidate information from various data sources, ensuring that person entities are stored under a single unique identifier.


In [0]:
# 👾 INPUTS 👾
PORCH_DATA = 'development.silver.dim_porch_current_contacts'
ACXIOM_DATA = 'development.silver.dim_acxiom_current_contacts'
BLUEPRINT_DATA = 'development.silver.blueprint_contacts'

# 👾 OUTPUTS 👾
MODEL = '/dbfs/FileStore/splink_saved_json_files/entity_resolution_model_train.json'
PREDICTION_TABLE = 'development.splink_training_data.entity_resolution_training_predictions'
CLUSTER_TABLE = 'development.splink_training_data.entity_resolution_training_clusters'

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

##### 👾  Training Data  👾

Training data is cleaned & prepared according to our [pre-processing requirements](https://dbc-cc8b3cdd-9625.cloud.databricks.com/?o=3471402352352814#notebook/3188833296771907/command/3188833296771908). 

**Training Data Considerations** </br>
 ```
💥 Data should be a random sample of the population. Check for bias introduced through your sampling technique.
💥 The Expectation Maximisation Algorithm works best when the pariwise record comparisons are a mix of between 0.1% and 0.99% true matches.
💥 Ensure that the dataset contains sufficient information to distinguish between records. 
💥 Your sample should represent the full range of variability in your dataset. 
```

In [0]:

query = f"""
SELECT a.*,
      coalesce(upper(nicknames_1.name1), first_name) as nickname1,
      coalesce(upper(nicknames_2.name1), first_name) as nickname2,
      coalesce(upper(nicknames_3.name1), first_name) as nickname3,
      coalesce(upper(nicknames_4.name1), first_name) as nickname4,
      soundex(first_name) as phonetic_first_name,
      soundex(last_name) as phonetic_last_name,
      soundex(full_name) as phonetic_full_name
FROM (
      SELECT 
        data_source,
        contact_id as unique_id, 
        full_name,
        first_name,
        last_name, 
        last_name_alt_2,
        nickname, 
        street_number,
        street_name, 
        unit_number,
        CONCAT(street_number,' ',street_name, ' ', coalesce(unit_number, '')) AS full_address,
        CONCAT(street_number,' ',street_name) AS address_1,
        zip5,
        age, 
        gender,
        birth_month,
        birth_day
      FROM {BLUEPRINT_DATA}
      WHERE error_code not in (311, 412, 112, 218, 220, 413, 215, 313, 216)
      UNION
      SELECT
        data_source,
        contact_id as unique_id, 
        a.full_name,
        a.first_name,
        a.last_name, 
        last_name_alt_2,
        nickname, 
        street_number,
        street_name, 
        unit_number,
        CONCAT(street_number,' ',street_name, ' ', coalesce(unit_number, '')) AS full_address,
        CONCAT(street_number,' ',street_name) AS address_1,
        zip5,
        case when age_missing = 1 then null else age end as age,
        case when gender_imputed = 1 then null when gender in ('A') then null else gender end as gender,
        case when birth_date_missing = 1 then null else MONTH(birth_date::date) end as birth_month,
        case when birth_date_missing = 1 then null else DAY(birth_date::date) end as birth_day
      FROM {PORCH_DATA} a
      JOIN development.silver.dim_porch_current on sequence = contact_id
      UNION
      SELECT
        data_source,
        contact_id as unique_id, 
        full_name,
        first_name,
        last_name, 
        last_name_alt_2,
        nickname, 
        street_number,
        street_name, 
        unit_number,
        CONCAT(street_number,' ',street_name, ' ', coalesce(unit_number, '')) AS full_address,
        CONCAT(street_number,' ',street_name) AS address_1,
        zip5,
        case when age_missing = 1 then null else age end as age,
        case when gender in ('A') then null else gender end as gender,
        null as birth_month,
        null as birth_day
      FROM {ACXIOM_DATA} a
      INNER JOIN development.silver.dim_acxiom_current on record_key=contact_id) a
left join main.sandbox.nickname_list_1 as nicknames_1 on (first_name == nicknames_1.name2)
left join main.sandbox.nickname_list_2 as nicknames_2 on (first_name == nicknames_2.name2)
left join  main.sandbox.nickname_list_3 as nicknames_3 on (first_name == nicknames_3.name2)
left join main.sandbox.nickname_list_4 as nicknames_4 on (first_name == nicknames_4.name2)
"""
df = sqlContext.sql(query)
display(df)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

#### EVIDENCE OF MATCH 
🤖`----------------`🤖`--------------------`🤖`--------------------`🤖 </br>



**👾  comparison vector design  👾**</br>
The comparison vector is our measure of similarity between two records (or evidence of a match). It is comprised of `comparisons` and `comparison levels`.


**Comparisons:** *what to compare?*  </br>
`comparisons` are the features we compare between records to find evidence for a match. </br>

```
💥 Choose features that provide meaningful evidence for distinguishing matches from non matches.
💥 Ensure each feature offers unique information. 
💥 Features should possess ambiguity; those that are always exact matches, like zip code, can bias the model.
💥 Do not compare correlated features
💥 Level of Missingness: features with high % missing are less useful
💥 Cardinality: features with higher cardinality are more useful
💥 Skew: a high cardinality feature where most data falls into one category is less useful 
💥 Combine features to create more robust representations and to deal with correlation, low cardinality, high proportion of nulls.
```

**Comparison Levels:** *how to compare?*  </br>
Each `comparison` has associated `levels` expressing a class or measure of similarity and thus, a different type (and amount) of evidence for a match. 

```
💥 Be familiar with the ambiguity in your data
💥 Start with similarity measures that are theoretically suitable to the comparison.
💥 Begin with few levels and gradually introduce complexity.
💥 Each comparison level should have a substantial amount of data. Remove overly specific levels.
💥 If a feature is skewed consider setting term_frequency_adjustment = True 
```
🧰 Validate similarity measures against examples: `ch.comparator_score("Richard", "iRchard")`


In [0]:
# Name Comparison 🏷️
comparison_name = {
    "output_column_name": "name",
    "comparison_description": "name",
    "comparison_levels": [ {"sql_condition": "full_name_l IS NULL OR full_name_r IS NULL", "label_for_charts": "Null", "is_null_level": True,},
                            # ✨ Full Name Exact Match 
                          {"sql_condition": "full_name_l = full_name_r", "label_for_charts": "Full Name Exact Match",},
                              # ✨ First Name Last Name Match 
                          {"sql_condition": "first_name_l = first_name_r and last_name_l = last_name_r","label_for_charts": "First Name Last Name Match",},
                          # ✨ First Name Hyphenated Last Name Match  
                          {"sql_condition": "first_name_l = first_name_r and (last_name_l = last_name_alt_2_r or last_name_r = last_name_alt_2_l)","label_for_charts": "Hyphenated Last Name Match",},
                          # ✨ Reverse First/Last Name Match 
                          {"sql_condition": "first_name_l = last_name_r and last_name_l = first_name_r", "label_for_charts": "Reverse Name Match",},
                            # ✨ Nickname Match 
                          {"sql_condition": "(nickname1_l = first_name_r or nickname1_r = first_name_l or nickname2_l = first_name_r or nickname2_r = first_name_l or nickname3_l = first_name_r or nickname3_r = first_name_l or nickname4_l = first_name_r or nickname4_r = first_name_l) and last_name_l = last_name_r",
                           "label_for_charts": "NickName Match",},
                            # ✨ Phonetic Full Name Match 
                          {"sql_condition": "(phonetic_first_name_l = phonetic_first_name_r and phonetic_last_name_l = phonetic_last_name_r) or (phonetic_full_name_l = phonetic_full_name_r and phonetic_full_name_l != phonetic_first_name_l and phonetic_full_name_l != phonetic_first_name_r  and jaro_winkler(`last_name_l`, `last_name_r`) > 0.9)", "label_for_charts": "Phonetic Full Name Match",},
                            # ✨ Jaro Winkler Match 
                          {"sql_condition": "jaro_winkler(`first_name_l`, `first_name_r`) > 0.85 and (jaro_winkler(`last_name_l`, `last_name_r`) > 0.9 or levenshtein(`last_name_l`, `last_name_r`) <= 2)", "label_for_charts": "Jaro Winkler",},
                            # ✨ First Name Middle Initial
                          {"sql_condition": "first_name_l = first_name_r","label_for_charts": "First Name Match",},
                          {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},],
}
   
# Address Comparison 🏠 
comparison_address = {
    "output_column_name": "address",
    "comparison_description": "address",
    "comparison_levels": [{"sql_condition": "full_address_l IS NULL OR full_address_r IS NULL", "label_for_charts": "Null", "is_null_level": True,},
                          # ✨ Full Address Exact Match 
                          {"sql_condition": "full_address_l = full_address_r", "label_for_charts": "Full Address Exact Match",},
                           # ✨ Missing address 2
                          {"sql_condition": "(address_1_l = address_1_r and unit_number_l is null) or (address_1_l = address_1_r and unit_number_r is null)", "label_for_charts": "Address 1 Exact Match missing unit number",},
                          {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},],
}

#  Age Comparison 👵🏻
comparison_age = {
    "output_column_name": "age",
    "comparison_description": "age",
    "comparison_levels": [{"sql_condition": "age_l is NULL or age_r is NULL", "label_for_charts": "Null", "is_null_level": True,},
                          # ✨ Age Exact Match 
                          {"sql_condition": "age_l = age_r" , "label_for_charts": "Age Exact Match",},
                          # ✨ Age within 2 years
                          {"sql_condition": "ABS(age_l - age_r)<=2", "label_for_charts": "Age within 2 years",},
                          # ✨ Age within 5 years
                          {"sql_condition": "ABS(age_l - age_r)<=5", "label_for_charts": "Age within 5 years",},
                          {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},],
}

# Birthdate Comparison 📅
comparison_birthdate = {
    "output_column_name": "birthdate",
    "comparison_description": "birthdate",
    "comparison_levels": [{"sql_condition": "birth_day_l is NULL or birth_day_r is NULL", "label_for_charts": "Null", "is_null_level": True,},
                          # ✨ BDaY Exact Match 
                          {"sql_condition": "birth_day_l = birth_day_r and birth_month_r = birth_month_l" , "label_for_charts": "birthday Exact Match",},
                          {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},],
    }

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'


**👾  comparison space  👾** </br>
We manage the computational cost of conducting pairwise comparisons by using `blocking rules` to subset our comparison space. These rules specify which pairs of records should be included for matching, with the primary goal of reducing the number of comparisons that the computer must perform. It is important to exercise caution and ensure that potential matches are not inadvertently excluded!
```
💥 Only eliminate records that are certain not to be a match
💥 Select features that effectively reduce the dataset size
💥 Use consistent and reliable features
💥 Avoid redundant blocking conditions 
```
🧰 check the size of your comparison space: `linker.cumulative_num_comparisons_from_blocking_rules_chart()`



In [0]:

# Configuration for comparison vector + comparison space

settings = {"comparisons": [comparison_name,
                            comparison_address,
                            comparison_birthdate,
                            cl.exact_match("gender"),
                            comparison_age],
            "link_type": "dedupe_only", 
            "unique_id_column_name": "unique_id",
            "blocking_rules_to_generate_predictions": [block_on(["address_1", "zip5"])],
            "retain_intermediate_calculation_columns": True,}
            
linker = SparkLinker(df, settings, spark=spark, set_up_basic_logging=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

#### MODEL TRAINING   
🤖`----------------`🤖`--------------------`🤖`--------------------`🤖 </br>

Training involves estimating the parameters that comprise Baye's formula to calculate the posterior probability that two records (a,b) are a match.


##### 👾 Parameter Estimation: λ 👾

The prior probability that two records drawn at random refer to the same entity `P(Records Match)`. </br> 
This is our match rate before considering any evidence. It should reflect the overall proportion of true matches in the dataset. </br>

**method**: deterministic match rule + recall (% of matches identified by the rule).</br>
```
💥 Deterministic Rule should be reliable and have a high degree of certainty.
💥 rules should not overly restrict the recall, as this could lead to missing true matches and underestimation of λ.
💥 Balance the use of rules with the goal of capturing all true matches (high recall). 
```
**interpretation**:</br>
* Higher λ: biases the model towards predicting more matches. May be desirable if capturing all true matches (high recall) is more important than minimizing false positives (low precision). May indicate datasets with more duplicates or a higher degree of redundancy
* Lower λ: biases the model towards predicting fewer matches. May be preferable if minimizing false positives is a higher priority than capturing all true matches. May suggest datasets with fewer duplicates or more distinct entities.


In [0]:
# Estimated λ: 
deterministic_rules = ['l.full_name = r.full_name and l.full_address = r.full_address and l.zip5=r.zip5']
linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.8)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

##### 👾 Parameter Estimation: u 👾

`P(Observation | Records Do Not Match)`. This is a measure of *coincidence*.

**method**: randomly sampling the dataset and observing how often a comparison level agreement occurs. </br> 
The probability of two random records being a match is very low so we can assume all pairs in the sample are not matches.


In [0]:
# Estimated u: 
linker.estimate_u_using_random_sampling(max_pairs=3e9)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'


##### 👾 Parameter Estimation: m 👾
`P(Observation | Records Match)`. This is the strength of evidence for a match.
</br>
It is a measure of *data quality/reliability*. How often might a persons information change legitimately (through time) or through data error?


**method**:  The Expectation-Maximization algorithm finds the values of \\(m\\) that best explain the observed evidence in the data by maximizing the likelihood function. 
1. Initialization: initialize \\(m\\) with some values
2. Expectation (E-step): calculate the probability of a match given the evidence \\(γ_i\\) and \\(m\\) .
3. Maximization (M-step): update \\(m\\) by maximizing the likelihood function L = \\(∏P(γ_i |m)\\) 
4. Iteration: repeat until \\(m\\) stabilizes (convergence)

The expectation maximisation algorithm is run on a subset of blocked record comparisons. 
These blocking rules can introduce bias if they disproportionately select certain types of record pairs.
```
💥 subset the comparison space by selecting pairs of records that are likely to refer to the same entity 
💥 Try not to bias the estimation by oversampling certain "types" of matches or non-matches.
```



In [0]:
#estimation of m for name 
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["address_1", "zip5", "age"]))
 

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for name 
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["full_address", "zip5"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for name 
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["full_address", "age", "zip5"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for name 
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["full_address", "birth_day", "birth_month", "zip5"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for address
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["first_name", "last_name", "zip5"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for age
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["first_name", "last_name", "full_address", "gender"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for gender
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["first_name", "full_address", "age"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
#estimation of m for gender + age
training_session_names =linker.estimate_parameters_using_expectation_maximisation(block_on(["full_name", "full_address"]))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'


##### 👾 Model Training Behavior 👾
The more stable a model is in the training process, the more reliable the outputs are.

`Stability within training session`</br>
Speed of convergence is shown in the terminal output during training (above). In general, the fewer iterations required to converge the better.

`Stability across training sessions`</br>
If the m estimates vary significantly from one run to another, it indicates instability in the estimation process or sensitivity to random variations in the data.
```
💥 Sensitivity to small changes in the features, blocking rules, or hyperparameters. 
💥 Data Quality Issues (missing values, errors, inconsistencies)
💥 Sample Variability can result in different patterns of agreement and disagreement 
💥 complex models may exhibit variability due to their flexibility. Look at similarity measures, model assumptions, and optimization algorithms
💥 Convergence Issues: failing to converge to a stable solution or getting stuck in local minima
💥 Overly complex models may overfit to noise. Overly simplistic models may result in biased or unreliable m estimates.
💥 Inconsistent initialization strategies or random initialization conditions 
```


In [0]:
linker.parameter_estimate_comparisons_chart()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

### PARAMETER EVALUATION
🤖`----------------`🤖`--------------------`🤖`--------------------`🤖 </br>


##### 👾 Parameter Interpretation: u 👾
\\(u\\)-values help establish a threshold for determining whether observed agreement between two records is sufficiently indicative of a true match. Records with agreement probabilities above the threshold are classified as potential matches, while those below the threshold are considered non-matches.
* Higher \\(u\\): higher likelihood of observing agreement between non-matching records & suggest that the model should be more conservative in predicting matches to avoid false positives.
* Lower \\(u\\): model can be more confident in predicting matches, as chance agreement between non-matching records is less likely.


```
💥 Highly driven by cardinality. two different people have the same gender much more often than last name
💥 Sparse data or low variability in comparison variables may pose challenges to accurate u estimation, leading to uncertainty in matching decisions.
```
##### 👾 Parameter Interpretation: m 👾
* Higher \\(m\\)-values: stronger likelihood of agreement between matching records, supporting the decision to classify them as matches. 
* Lower \\(m\\)-values: less agreement between matching records, which may require additional evidence or consideration before classifying them as matches.

```
Reasons for estimated m being too high
💥 The comparison variables have high agreement by chance. Can occur if they are not sufficiently discriminative or highly correlated.
💥 The comparison variables are sparse or have low variability, the model may overestimate the likelihood of matches
💥 If there are errors or inconsistencies in the data, the model may incorrectly assign high probabilities to non-matches
💥 If the comparison variables are correlated, the estimated m-probabilities may be biased upwards.

Reasons for estimated m-probabilities being too low:
💥 If the comparison variables exhibit high variability or noise, the model may underestimate the likelihood of matches
💥 comparison variables with limited samples or insufficient information
💥 If important variables or features that contribute to identifying matches are missing from the dataset, the model may underestimate the likelihood of matches.
💥 If the model is too simplistic and fails to capture the complexities of the data, it may underestimate the likelihood of matches, leading to lower m-probabilities.
```






In [0]:
linker.m_u_parameters_chart()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

##### 👾 Parameter Interpretation: Match Weight 👾
The Bayes Factor \\(\frac{m}{u}\\) is a ratio of two likelihoods, representing the relative evidence in favor of a match compared to a non match. It provides a quantitative measure of the strength of evidence supporting one hypothesis over another.
K acts as a relative multiplier that increases or decreases the overall prediction of whether the records match. </br>
K = 5 ⇢`5x more likely to match` We observe this scenario around 5 times more often among matching records than matching records.</br>
K = 0.2 ⇢ `5x less likely to match` We observe this scenario around 5 times more often among non-matching records than matching records. </br>

strong positive match weights are only possible with low `u`, implying high cardinality. 
strong negative match weights only possible with low `m`, which in turn implies high data quality.</br>

In [0]:
# Visualizing Model Parameters
linker.match_weights_chart()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

### MODEL OUTPUTS
🤖`----------------`🤖`--------------------`🤖`--------------------`🤖 </br>

##### 👾 Save Model 👾

In [0]:
# 💾 save model
settings = linker.save_model_to_json(f'{MODEL}', overwrite=True)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

##### 👾 Model Predictions 👾

In [0]:
# 👾 GENERATE PREDICTIONS 👾
predictions = linker.predict(threshold_match_probability=0.2)
df_predictions = predictions.as_spark_dataframe()
display(predictions.as_pandas_dataframe(limit=10000))
# 💾 save predictions
df_predictions.write.format("delta").mode("overwrite").saveAsTable(f"{PREDICTION_TABLE}")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
records_to_view  = predictions.as_record_dict(limit=50)
linker.waterfall_chart(records_to_view, filter_nulls=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'

In [0]:
# 👾 GENERATE CLUSTERS 👾
clusters = linker.cluster_pairwise_predictions_at_threshold(predictions, threshold_match_probability=0.2)
clusters_df = clusters.as_spark_dataframe()
# 💾 save clusters
clusters_df.write.format("delta").mode("overwrite").saveAsTable(f"{CLUSTER_TABLE}")


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2143697984260545>, line 2
      1 # 👾 IMPORTS 👾
----> 2 from splink.spark.jar_location import similarity_jar_location
      3 from pyspark import SparkContext, SparkConf
      4 from pyspark.sql import SparkSession, types

ModuleNotFoundError: No module named 'splink.spark'